In this notebook we will check for inconsistencies in the dataset:
- prices between no-arbitrage bounds
- put-call-parity
- negative time values

In [1]:
@time include("../startup_script.jl");

elapsed time: 0.611456806 seconds (52396500 bytes allocated, 5.23% gc time)
elapsed time: 59.982242506 seconds (5419744236 bytes allocated, 61.33% gc time)
elapsed time: 0.84185482 seconds (112671656 bytes allocated, 43.65% gc time)
elapsed time: 73.316054299 seconds (5419744236 bytes allocated, 68.39% gc time)
elapsed time: 160.434629087 seconds (14793848892 bytes allocated, 56.26% gc time)


### Put-Call parity

- following Hull, we have:

$$c+Ke^{-rT}=p+S_{0}$$

### Bounds

- for non-dividend-paying stocks we get the following bounds

$$\max (S_{0} - Ke^{-rT}, 0) \leq c \leq S_{0}$$
$$ \max (Ke^{-rT} - S_{0}, 0) \leq p \leq Ke^{-rT}$$

- create required data matrix

In [3]:
consistData = join(optPrices, opts, on = :ID) |>
x -> join(x, cohortParams, on = [:Date, :Expiry]) |>
x -> join(x, daxVals, on = :Date)

head(consistData)

,Date,ID,Price,Expiry,Strike,IsCall,EONIA_matched,Time_to_Maturity,DAX
1,2006-07-03,c_20060721_4500,1212.0,2006-07-21,4500,true,0.0283102283088403,0.0549019607843137,5712.69
2,2006-07-03,c_20060721_4600,1112.3,2006-07-21,4600,true,0.0283102283088403,0.0549019607843137,5712.69
3,2006-07-03,c_20060721_4700,1012.7,2006-07-21,4700,true,0.0283102283088403,0.0549019607843137,5712.69
4,2006-07-03,c_20060721_4800,913.2,2006-07-21,4800,true,0.0283102283088403,0.0549019607843137,5712.69
5,2006-07-03,c_20060721_4850,863.5,2006-07-21,4850,true,0.0283102283088403,0.0549019607843137,5712.69
6,2006-07-03,c_20060721_4900,813.9,2006-07-21,4900,true,0.0283102283088403,0.0549019607843137,5712.69


In [9]:
function callLowBound(s::Float64, k::Int, r::Float64, T::Float64)
    return maximum([s - k*exp(-r*T), 0])
end

function putLowBound(s::Float64, k::Int, r::Float64, T::Float64)
    return maximum([k*exp(-r*T - s), 0])
end

putLowBound (generic function with 1 method)

In [14]:
function priceBounds(x::DataFrame)
    nObs = size(x, 1)
    lowBounds, upBounds = (zeros(nObs), zeros(nObs))
    for ii=1:nObs
        if x[ii, :IsCall]
            lowBounds[ii] = callLowBound(x[ii, :DAX], x[ii, :Strike], x[ii, :EONIA_matched], x[ii, :Time_to_Maturity])
            upBounds[ii] = x[ii, :DAX]
        else
            lowBounds[ii] = putLowBound(x[ii, :DAX], x[ii, :Strike], x[ii, :EONIA_matched], x[ii, :Time_to_Maturity])
            upBounds[ii] = x[ii, :Strike]*exp(-x[ii, :EONIA_matched]*x[ii, :Time_to_Maturity])
        end
    end
    return (lowBounds, upBounds)         
end            

priceBounds (generic function with 1 method)

In [15]:
lowBounds, upBounds = priceBounds(consistData)

([1219.68,1119.83,1019.99,920.145,870.222,820.3,770.378,720.455,670.533,620.611  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[5712.69,5712.69,5712.69,5712.69,5712.69,5712.69,5712.69,5712.69,5712.69,5712.69  …  9199.56,9249.29,9348.75,9398.47,9448.2,9497.93,9547.65,9647.11,9746.56,9846.02])

In [17]:
consistData[:LowBound] = lowBounds
consistData[:UpBound] = upBounds

head(consistData)

,Date,ID,Price,Expiry,Strike,IsCall,EONIA_matched,Time_to_Maturity,DAX,LowBound,UpBound
1,2006-07-03,c_20060721_4500,1212.0,2006-07-21,4500,true,0.0283102283088403,0.0549019607843137,5712.69,1219.678858946394,5712.69
2,2006-07-03,c_20060721_4600,1112.3,2006-07-21,4600,true,0.0283102283088403,0.0549019607843137,5712.69,1119.8341669229812,5712.69
3,2006-07-03,c_20060721_4700,1012.7,2006-07-21,4700,true,0.0283102283088403,0.0549019607843137,5712.69,1019.9894748995675,5712.69
4,2006-07-03,c_20060721_4800,913.2,2006-07-21,4800,true,0.0283102283088403,0.0549019607843137,5712.69,920.1447828761538,5712.69
5,2006-07-03,c_20060721_4850,863.5,2006-07-21,4850,true,0.0283102283088403,0.0549019607843137,5712.69,870.2224368644474,5712.69
6,2006-07-03,c_20060721_4900,813.9,2006-07-21,4900,true,0.0283102283088403,0.0549019607843137,5712.69,820.3000908527401,5712.69


- frequency of option prices below option lower bound

In [18]:
sum(consistData[:Price] .< consistData[:LowBound])/size(consistData, 1)

0.031203444323793694

- frequency of option prices above upper bound

In [19]:
sum(consistData[:Price] .> consistData[:UpBound])/size(consistData, 1)

0.0

- checking put-call parity